# 🚀 鳳凰之心 - Colab 儀表板端對端測試

歡迎使用鳳凰之心專案的 Google Colab 測試與部署儀表板。這個 Notebook 將引導您完成從下載、測試到啟動服務的完整流程。

**請按照以下步驟依序執行每個程式碼區塊。**

## 步驟 1：準備環境並下載專案

這個區塊會做幾件事：
1.  從 GitHub 上複製最新的專案程式碼。
2.  切換到專案的目錄中。
3.  安裝我們測試腳本所需要的核心依賴 (`uv`, `psutil`, `pyyaml`)。

In [ ]:
import os
import subprocess

# 定義專案名稱
project_name = "phoenix-heart-project"

# 1. 複製專案
if not os.path.exists(project_name):
    print(f"📂 正在從 GitHub 複製專案...")
    # 注意：這裡使用了一個假設的 URL，請在實際使用時替換成您的專案 URL
    subprocess.run(["git", "clone", f"https://github.com/your-username/{project_name}.git"], check=True)
else:
    print(f"✅ 專案目錄 '{project_name}' 已存在，跳過複製。")

# 2. 切換目錄
os.chdir(project_name)
print(f"getcwd: {os.getcwd()}")

# 3. 安裝核心依賴
print("\n🐍 正在安裝核心 Python 依賴...")
subprocess.run(["pip", "install", "-q", "uv", "psutil", "pyyaml"], check=True)
print("✅ 核心依賴安裝完成！")

## 步驟 2：執行完整的端對端測試

現在，我們將執行 `smart_e2e_test.py` 腳本。這個腳本會自動完成以下所有工作：

- **平行化測試**：同時測試 `quant` 和 `transcriber` 兩個應用。
- **隔離環境**：為每個應用建立獨立的虛擬環境。
- **安全安裝**：在安裝依賴前檢查系統資源。
- **自動化測試**：使用 `pytest`、`pytest-xdist` 和 `pytest-timeout` 執行所有測試。

這個過程會需要幾分鐘，請耐心等候。

In [ ]:
print("🧪 正在啟動端對端測試...")
# 我們將以 `real` 模式執行，以確保所有依賴都被測試
test_env = os.environ.copy()
test_env["TEST_MODE"] = "real"

result = subprocess.run(["python", "smart_e2e_test.py"], env=test_env, capture_output=True, text=True)

print("--- STDOUT ---")
print(result.stdout)
print("--- STDERR ---")
print(result.stderr)

if result.returncode == 0:
    print("\n✅ 端對端測試成功通過！")
else:
    print("\n❌ 端對端測試失敗。請檢查上面的日誌輸出。")

## 步驟 3：在背景啟動所有後端服務

測試通過後，我們現在可以啟動所有後端微服務了。我們將使用 `launch.py` 腳本，它會：

1.  為每個應用建立**正式的**虛擬環境。
2.  安裝所有**正式的**依賴。
3.  在背景啟動每個服務的 FastAPI 伺服器。
4.  啟動一個逆向代理，將所有流量路由到正確的服務。

**注意**：這個儲存格會持續運行以保持服務在背景執行。您可以在下一個儲存格中與服務互動。

In [ ]:
import asyncio
from multiprocessing import Process
import nest_asyncio

nest_asyncio.apply() # 允許在 Colab 中巢狀執行 asyncio 事件循環

def run_launcher():
    # 導入 launch.py 中的 main 函數並執行它
    # 這種方式比 subprocess 更好，因為它可以更好地控制
    try:
        from launch import main as launch_main
        asyncio.run(launch_main())
    except Exception as e:
        print(f"啟動服務時發生錯誤: {e}")

print("🚀 正在背景啟動所有服務...")
server_process = Process(target=run_launcher)
server_process.start()
print("✅ 所有服務應該已經在背景啟動。請等待幾秒鐘讓它們完全初始化。")

# 為了確保服務有足夠的時間啟動，我們等待一下
import time
time.sleep(15) # 等待 15 秒

## 步驟 4：驗證儀表板是否正常運行

現在，所有服務都應該在背景運行了。最後一步是驗證我們的儀表板是否可以成功訪問。

這個儲存格會：
1.  向 `phoenix_starter.py` 視覺化啟動器（假設它提供一個Web儀表板）的預期地址發送一個 HTTP GET 請求。
2.  檢查 HTTP 回應狀態碼是否為 `200 OK`。
3.  檢查回應的 HTML 內容中是否包含特定的標題文字，例如 `"鳳凰之心指揮中心"`，以確認是正確的頁面。

**注意**：此處的驗證目標是 `phoenix_starter.py` 的儀表板，而不是 `launch.py` 啟動的 API。這是一個假設，需要根據 `phoenix_starter.py` 的實際功能進行調整。如果儀表板是透過不同的方式提供，需要修改下面的 URL。

In [ ]:
import httpx

# 假設 phoenix_starter 的儀表板運行在 8000 埠上
# 這是根據 proxy_config.json 的設定
dashboard_url = "http://localhost:8000"
expected_title = "鳳凰之心指揮中心" # 假設儀表板 HTML 中的 <title> 或 <h1>

print(f"🔍 正在嘗試訪問儀表板: {dashboard_url}")

try:
    with httpx.Client() as client:
        response = client.get(dashboard_url)
        
        print(f"📊 HTTP 狀態碼: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ 成功收到回應！")
            # 由於 phoenix_starter.py 是 TUI 應用，它不會提供 HTTP 服務。
            # 我們將改為驗證由 launch.py 啟動的 proxy。
            # 我們預期直接訪問根路徑會得到一個路由錯誤，這也是一種驗證。
            if "無法路由的請求" in response.text:
                 print("✅ 成功驗證逆向代理的預設行為！")
            else:
                 print("⚠️ 代理的回應非預期，但至少服務在運行。")
                 print(f"回應內容 (前100字元): {response.text[:100]}")
        else:
            print(f"❌ 儀表板訪問失敗。狀態碼: {response.status_code}")

except httpx.ConnectError as e:
    print(f"❌ 連接錯誤: 無法連接到 {dashboard_url}。")
    print("   請確認上一步的服務是否仍在背景正常運行。")
except Exception as e:
    print(f"❌ 發生未預期的錯誤: {e}")

# 最後，終止背景服務
print("\n🛑 正在終止背景服務...")
server_process.terminate()
server_process.join()
print("✅ 所有服務已清理完畢。")